In [ ]:
import numpy as np
import pandas as pd
import torch
from torch import nn
import torch.optim as optim
from tqdm import tqdm

# =============================================================================
# 0. Fixed Hyperparameters for Ablation
# =============================================================================
CSV_IN        = "dataset.csv"
CSV_OUT       = "ablation_results.csv"

LOOK_BACK     = 10
WIN_SIZE      = 4
OVERLAP       = 0.75
WIN_STEP      = max(1, int(WIN_SIZE * (1 - OVERLAP)))
N_WINDOWS     = (LOOK_BACK - WIN_SIZE) // WIN_STEP + 1
INPUT_FEATURES_NUM = N_WINDOWS * 8 + 1  # 8 stats per window + effect

HIDDEN_SIZE   = 8
OUTPUT_FEATURES_NUM = 1
NUM_LAYERS    = 1
MAX_EPOCHS    = 1000
LEARNING_RATE = 0.05
CIRCLE        = 5

# Prepare results file
cols = ["scenario","LOOK_BACK","WIN_SIZE","OVERLAP","WIN_STEP","N_WINDOWS","MSE","RMSE","RAE","R2"]
pd.DataFrame(columns=cols).to_csv(CSV_OUT, index=False)

# =============================================================================
# 1. Data Import
# =============================================================================
def import_TSD_data(filename=CSV_IN):
    df = pd.read_csv(filename)
    trend  = df["rate"].astype("float32").values
    cycle  = df["cycle"].astype("float32").values
    effect = df["indicator_norm"].astype("float32").values
    train_size = int(0.6 * len(trend))
    return trend, cycle, effect, train_size

trend_orig, cycle_orig, effect_orig, train_size = import_TSD_data()

# =============================================================================
# 2. Feature Builder (stats + effect), with separate target array
# =============================================================================
def build_stats_features(trend_feat, cycle, effect,
                         look_back, win_size, win_step,
                         trend_target=None):
    target_offset = 2 * look_back
    n_samples = len(trend_feat) - (look_back + target_offset)
    X, y = [], []
    for i in range(n_samples):
        r_seg = trend_feat[i : i + look_back]
        c_seg = cycle    [i : i + look_back]
        feats = []
        for w in range(0, look_back - win_size + 1, win_step):
            r_win = r_seg[w : w + win_size]
            c_win = c_seg[w : w + win_size]
            feats.extend([
                r_win.min(), r_win.max(), r_win.mean(), r_win.std(ddof=0),
                c_win.min(), c_win.max(), c_win.mean(), c_win.std(ddof=0)
            ])
        feats.append(effect[i + target_offset])
        X.append(feats)
        # choose the target array: default to trend_feat if none passed
        tgt = trend_target if trend_target is not None else trend_feat
        y.append(tgt[i + target_offset])
    return np.array(X, dtype="float32"), np.array(y, dtype="float32")

# =============================================================================
# 3. GRU Model Definition
# =============================================================================
class GRU_model(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers):
        super().__init__()
        self.gru = nn.GRU(input_size - 1, hidden_size, num_layers)
        self.head = nn.Linear(hidden_size + 1, output_size)

    def forward(self, x):
        # x: (seq_len, batch, input_size)
        seq, batch, _ = x.shape
        data = x[:, :, :INPUT_FEATURES_NUM - 1]   # time‑series part
        eff  = x[:, :, -1].unsqueeze(2)           # effect feature
        out, _ = self.gru(data)                   # (seq_len, batch, hidden)
        out = torch.cat([out, eff], dim=2).view(seq * batch, -1)
        out = self.head(out)
        return out.view(seq, batch, -1)

# =============================================================================
# 4. Train & Evaluate Function (accepts separate feature & target trends)
# =============================================================================
def TSD_GRU(trend_feat, cycle, effect, trend_target,
            train_size, look_back, win_size, win_step):
    # split feature vs target arrays
    tr_feat, te_feat = trend_feat[:train_size], trend_feat[train_size:]
    tr_tar,  te_tar  = trend_target[:train_size], trend_target[train_size:]
    tr_cy,  te_cy   = cycle[:train_size],    cycle[train_size:]
    tr_eff, te_eff  = effect[:train_size],   effect[train_size:]

    # build stats features and correct targets
    X_tr, y_tr = build_stats_features(tr_feat, tr_cy, tr_eff,
                                      look_back, win_size, win_step,
                                      trend_target=tr_tar)
    X_te, y_te = build_stats_features(te_feat, te_cy, te_eff,
                                      look_back, win_size, win_step,
                                      trend_target=te_tar)

    bs_tr, bs_te = X_tr.shape[0], X_te.shape[0]
    feat_dim     = X_tr.shape[1]

    # shape for GRU: (seq_len=1, batch, features)
    X_tr_t = torch.from_numpy(X_tr).reshape(1, bs_tr, feat_dim)
    y_tr_t = torch.from_numpy(y_tr).reshape(1, bs_tr, OUTPUT_FEATURES_NUM)
    X_te_t = torch.from_numpy(X_te).reshape(1, bs_te, feat_dim)
    y_te_t = torch.from_numpy(y_te).reshape(1, bs_te, OUTPUT_FEATURES_NUM)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = GRU_model(feat_dim, HIDDEN_SIZE,
                      OUTPUT_FEATURES_NUM, NUM_LAYERS).to(device)
    opt    = optim.Adam(model.parameters(), lr=LEARNING_RATE)
    loss_fn= nn.MSELoss()

    MSEs, RMSEs, RAEs, R2s = [], [], [], []

    for _ in range(CIRCLE):
        model.apply(lambda m: m.reset_parameters()
                    if hasattr(m, "reset_parameters") else None)
        # training
        for _ in range(MAX_EPOCHS):
            opt.zero_grad()
            out = model(X_tr_t.to(device))
            loss = loss_fn(out, y_tr_t.to(device))
            loss.backward()
            opt.step()
        # evaluation
        with torch.no_grad():
            pred = model(X_te_t.to(device)).cpu().view(-1).numpy()
        actual = y_te_t.view(-1).numpy()
        err = pred - actual

        mse = (err**2).mean()
        MSEs.append(mse)
        RMSEs.append(np.sqrt(mse))
        # safe RAE
        denom = np.sum(np.abs(actual - actual.mean()))
        rae   = np.sum(np.abs(err)) / denom if denom>0 else 0.0
        RAEs.append(rae)
        # safe R2
        RSS = np.sum(err**2)
        TSS = np.sum((actual - actual.mean())**2)
        r2   = 1 - RSS/TSS if TSS>0 else 1.0
        R2s.append(r2)

    return np.mean(MSEs), np.mean(RMSEs), np.mean(RAEs), np.mean(R2s)

# =============================================================================
# 5. Ablation Scenarios & Loop
# =============================================================================
scenarios = {
    "all_features": (trend_orig, cycle_orig, effect_orig),
    "no_trend":     (np.zeros_like(trend_orig), cycle_orig, effect_orig),
    "no_cycle":     (trend_orig, np.zeros_like(cycle_orig), effect_orig),
    "no_effect":    (trend_orig, cycle_orig, np.zeros_like(effect_orig)),
}

for name, (t_feat, c_arr, e_arr) in scenarios.items():
    mse, rmse, rae, r2 = TSD_GRU(
        t_feat,       # trend for features
        c_arr,
        e_arr,
        trend_orig,   # true trend for targets
        train_size,
        LOOK_BACK, WIN_SIZE, WIN_STEP
    )
    row = {
        "scenario":  name,
        "LOOK_BACK": LOOK_BACK,
        "WIN_SIZE":  WIN_SIZE,
        "OVERLAP":   OVERLAP,
        "WIN_STEP":  WIN_STEP,
        "N_WINDOWS": N_WINDOWS,
        "MSE":       mse,
        "RMSE":      rmse,
        "RAE":       rae,
        "R2":        r2
    }
    pd.DataFrame([row]).to_csv(CSV_OUT, mode='a', header=False, index=False)

# 6. Print results
print(pd.read_csv(CSV_OUT))
